# Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/uni_bonn/nlp/final_project

/content/drive/MyDrive/uni_bonn/nlp/final_project


# Install modules

In [ ]:
!pip install datasets
!pip install 'transformers[torch]'
!pip install torchmetrics
!pip install nltk sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 3.7 MB/s eta 0:00:00


# Import modules

In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import numpy as np
import json
from transformers import EncoderDecoderModel
import torchmetrics
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
import gc
import torch

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
import math
import sacrebleu
import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk import ngrams
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# Set model

In [ ]:
model_name = "patrickvonplaten/bert2bert-cnn_dailymail-fp16"

# Load dataset using Transformer datasets

In [ ]:
data_files={
    "train":"./dataset/GYAFC_Corpus/data_train.json",
    "val":"./dataset/GYAFC_Corpus/data_val.json",
    "test":"./dataset/GYAFC_Corpus/data_test.json",
}

In [ ]:
dataset = load_dataset("json", data_files=data_files)

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/3.22k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

In [ ]:
encoder_max_length=64
decoder_max_length=64

def preprocess_function(examples, input_field="informal", target_field="formal.ref0"):
  inputs = [ex[input_field] for ex in examples["transformation"]]
  targets = [ex[target_field] for ex in examples["transformation"]]

  inputs = tokenizer(inputs, text_target=targets, padding="max_length", truncation=True, max_length=encoder_max_length)
  outputs = tokenizer(targets, padding="max_length", truncation=True, max_length=decoder_max_length)

  examples["input_ids"] = inputs.input_ids
  examples["attention_mask"] = inputs.attention_mask
  examples["decoder_input_ids"] = outputs.input_ids
  examples["decoder_attention_mask"] = outputs.attention_mask
  examples["labels"] = outputs.input_ids.copy()

  # because BERT automatically shifts the labels, the labels correspond exactly to `decoder_input_ids`.
  # We have to make sure that the PAD token is ignored
  # replace padding token id's of the labels by -100 so it's ignored by the loss
  examples["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in examples["labels"]]

  return examples

In [ ]:
def create_multi_ref_dataset(dataset):
  for i, target_field in enumerate(['formal.ref0', 'formal.ref1', 'formal.ref2', 'formal.ref3']):
    new_dataset = dataset.map(
      preprocess_function,
      batched=True,
    )

    if i == 0:
      dataset = dataset.add_column('input_ids', new_dataset['input_ids'])
      # dataset = dataset.add_column('token_type_ids', new_dataset['token_type_ids'])
      dataset = dataset.add_column('attention_mask', new_dataset['attention_mask'])
      dataset = dataset.add_column('decoder_input_ids', new_dataset['decoder_input_ids'])
      dataset = dataset.add_column('decoder_attention_mask', new_dataset['decoder_attention_mask'])
      dataset = dataset.add_column('labels', new_dataset['labels'])
    else:
      dataset = dataset.add_column(f'labels_{i}', new_dataset['labels'])

  return dataset

In [ ]:
val_dataset = create_multi_ref_dataset(dataset['val'])
test_dataset = create_multi_ref_dataset(dataset['test'])

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/5665 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

Map:   0%|          | 0/2748 [00:00<?, ? examples/s]

In [ ]:
train_dataset = dataset['train'].map(
    preprocess_function,
    batched=True,
)

Map:   0%|          | 0/104562 [00:00<?, ? examples/s]

In [ ]:
print(val_dataset)
print(train_dataset)

Dataset({
    features: ['id', 'transformation', 'topic', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels', 'labels_1', 'labels_2', 'labels_3'],
    num_rows: 5665
})
Dataset({
    features: ['id', 'transformation', 'topic', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'labels'],
    num_rows: 104562
})


In [ ]:
print(train_dataset[0])

{'id': 0, 'transformation': {'formal.ref0': 'I prefer to let the guy ask me.', 'formal.ref1': '', 'formal.ref2': '', 'formal.ref3': '', 'informal': "Sure, it's ok, but I always have let the guy ask me."}, 'topic': 'Family_Relationships', 'input_ids': [101, 2469, 1010, 2009, 1005, 1055, 7929, 1010, 2021, 1045, 2467, 2031, 2292, 1996, 3124, 3198, 2033, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_input_ids': [101, 1045, 9544, 2000, 2292, 1996, 3124, 3198, 2033, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'decoder_attention_mask': [1, 1, 1, 1, 1, 

In [ ]:
tokenizer.decode(train_dataset[1708]["input_ids"], skip_special_tokens=True)

'besides how old were u when u lost virginity and the tables turn on u now!! ( lol )!'

# Finetune Bert2Bert Model

## Metrics implementations

In [ ]:
#Higher is better
def calculate_terp(hypothesis: str, reference: str, phrase_table=list(), edit_costs=list()):
    hypothesis_tokens = hypothesis.split()
    reference_tokens = reference.split()

    # TERp by Stem Matches, Synonym Matches, and Phrase Substitutions
    stem_matches = calculate_stem_matches(hypothesis_tokens, reference_tokens)
    synonym_matches = calculate_synonym_matches(hypothesis_tokens, reference_tokens)
    phrase_substitutions = calculate_phrase_substitutions(hypothesis_tokens, reference_tokens, phrase_table, edit_costs)

    # Calculate TERp score
    terp_score = (stem_matches + synonym_matches + phrase_substitutions) / len(reference_tokens)

    return terp_score

def calculate_stem_matches(hypothesis_tokens, reference_tokens):
    stemmer = PorterStemmer()
    stem_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                      if stemmer.stem(hyp_token.lower()) == stemmer.stem(ref_token.lower()))
    return stem_matches

def calculate_synonym_matches(hypothesis_tokens, reference_tokens):
    synonym_matches = sum(1 for hyp_token, ref_token in zip(hypothesis_tokens, reference_tokens)
                          if are_synonyms(hyp_token.lower(), ref_token.lower()))
    return synonym_matches

def are_synonyms(word1, word2):
    synsets1 = wordnet.synsets(word1)
    synsets2 = wordnet.synsets(word2)

    return any(set1.wup_similarity(set2) > 0.7 for set1 in synsets1 for set2 in synsets2)

def calculate_phrase_substitutions(hypothesis_tokens, reference_tokens, phrase_table, edit_costs):
    substitution_cost = 0

    for i in range(len(hypothesis_tokens)):
        for j in range(len(reference_tokens)):
            if (hypothesis_tokens[i], reference_tokens[j]) in phrase_table:
                # Retrieve paraphrase information from the phrase table
                paraphrase_info = phrase_table[(hypothesis_tokens[i], reference_tokens[j])]

                # Calculate the cost using the provided formula
                cost = (
                    edit_costs['w1'] +
                    edit_costs['w2'] * paraphrase_info['edit'] * math.log(paraphrase_info['probability']) +
                    edit_costs['w3'] * paraphrase_info['edit'] * paraphrase_info['probability'] +
                    edit_costs['w4'] * paraphrase_info['edit']
                )

                # Ensure the substitution cost is not negative
                substitution_cost += max(0, cost)

    return substitution_cost

def terp_alignment(hypothesis, reference, phrase_table=list(), edit_costs=list()):
    alignment = []

    for hyp_token, ref_token in zip(hypothesis.split(), reference.split()):
        if hyp_token == ref_token:
            alignment.append((hyp_token, ref_token, "Exact Match"))
        else:
            alignment.append((hyp_token, ref_token, "Mismatch"))

    return alignment

In [ ]:
def calculate_pinc(hypothesis: str, reference: str, n: int):
    hypothesis_split = hypothesis.split()
    reference_split = reference.split()

    hypothesis_ngrams = set(ngrams(hypothesis_split, n))
    reference_ngrams = set(ngrams(reference_split, n))
    new_ngrams = hypothesis_ngrams - reference_ngrams
    pinc_score = len(new_ngrams) / len(hypothesis_ngrams)

    return pinc_score

In [ ]:
# Example usage:
hypothesis_sentence = "This is an example sentence."
reference_sentence = "This is an example sentence."

# Calculate TERp score
terp_score = calculate_terp(hypothesis_sentence, reference_sentence)
print(f"TERp Score: {terp_score}")

# Generate alignment
alignment = terp_alignment(hypothesis_sentence, reference_sentence)
print("Alignment:", alignment)

TERp Score: 1.6
Alignment: [('This', 'This', 'Exact Match'), ('is', 'is', 'Exact Match'), ('an', 'an', 'Exact Match'), ('example', 'example', 'Exact Match'), ('sentence.', 'sentence.', 'Exact Match')]


In [ ]:
pinc_score = calculate_pinc(hypothesis_sentence, reference_sentence, 2)
print(f"PINC Score: {pinc_score}")

PINC Score: 0.0


In [ ]:
def terp(preds, refs):
  scores = np.zeros(len(preds), dtype=float)
  for i in range(len(preds)):
    pred = preds[i]
    ref = refs[i]
    score = np.min(np.array(list(map(lambda x: calculate_terp(pred, x), ref))))
    scores[i] = score

  return np.mean(scores)

In [ ]:
def pinc(preds, refs, n=2):
  scores = np.zeros(len(preds), dtype=float)
  for i in range(len(preds)):
    pred = preds[i]
    ref = refs[i]
    score = np.min(np.array(list(map(lambda x: calculate_pinc(pred, x, n), ref))))
    scores[i] = score

  return np.mean(scores)

## Train model

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pytorch_model.bin:   0%|          | 0.00/992M [00:00<?, ?B/s]

Some weights of the model checkpoint at patrickvonplaten/bert2bert-cnn_dailymail-fp16 were not used when initializing EncoderDecoderModel: ['decoder.bert.pooler.dense.bias', 'decoder.bert.pooler.dense.weight']
- This IS expected if you are initializing EncoderDecoderModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EncoderDecoderModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model.config.decoder_start_token_id = tokenizer.cls_token_id
model.config.eos_token_id = tokenizer.sep_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.vocab_size = model.config.encoder.vocab_size

model.config.max_length = 64
model.config.no_repeat_ngram_size = 3
model.config.early_stopping = True
model.config.length_penalty = 2.0
model.config.num_beams = 4

In [ ]:
print(model.config)

EncoderDecoderConfig {
  "_name_or_path": "patrickvonplaten/bert2bert-cnn_dailymail-fp16",
  "architectures": [
    "EncoderDecoderModel"
  ],
  "decoder": {
    "_name_or_path": "",
    "add_cross_attention": true,
    "architectures": [
      "BertForMaskedLM"
    ],
    "attention_probs_dropout_prob": 0.1,
    "bad_words_ids": null,
    "begin_suppress_tokens": null,
    "bos_token_id": null,
    "chunk_size_feed_forward": 0,
    "classifier_dropout": null,
    "cross_attention_hidden_size": null,
    "decoder_start_token_id": null,
    "diversity_penalty": 0.0,
    "do_sample": false,
    "early_stopping": false,
    "encoder_no_repeat_ngram_size": 0,
    "eos_token_id": null,
    "exponential_decay_length_penalty": null,
    "finetuning_task": null,
    "forced_bos_token_id": null,
    "forced_eos_token_id": null,
    "gradient_checkpointing": false,
    "hidden_act": "gelu",
    "hidden_dropout_prob": 0.1,
    "hidden_size": 768,
    "id2label": {
      "0": "LABEL_0",
      "1":

In [ ]:
train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels"],
)

In [ ]:
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", 'labels_1', 'labels_2', 'labels_3'],
)
test_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "decoder_input_ids", "decoder_attention_mask", "labels", 'labels_1', 'labels_2', 'labels_3'],
)

In [ ]:
def compute_metrics(pred):
  # labels_ids = pred.label_ids
  # pred_ids = np.argmax(np.array(pred.predictions[0]), axis=-1)
  labels_ids = pred.label_ids
  pred_ids = pred.predictions

  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  preds = [" ".join([token.text for token in nlp(line)]) for line in pred_str]
  labels = [[" ".join([token.text for token in nlp(line)]), ] for line in label_str]

  # Calculate metrics
  bleu1 = torchmetrics.text.BLEUScore(n_gram=1)
  bleu2 = torchmetrics.text.BLEUScore(n_gram=2)
  bleu3 = torchmetrics.text.BLEUScore(n_gram=3)
  bleu4 = torchmetrics.text.BLEUScore(n_gram=4)
  sacrebleu = torchmetrics.text.SacreBLEUScore()
  ter = torchmetrics.text.TranslationEditRate(normalize=True, lowercase=False)

  bleu1(preds, labels)
  bleu2(preds, labels)
  bleu3(preds, labels)
  bleu4(preds, labels)
  ter(preds, labels)
  sacrebleu(preds, labels)

  return dict(
      bleu1=bleu1.compute(),
      bleu2=bleu2.compute(),
      bleu3=bleu3.compute(),
      bleu4=bleu4.compute(),
      ter=ter.compute(),
      sacrebleu=sacrebleu.compute(),
      terp=terp(preds, labels),
      # pinc=pinc(preds, labels, n=4)
  )

In [ ]:
training_args = Seq2SeqTrainingArguments(
    evaluation_strategy="epoch",
    # eval_steps=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    output_dir="./model_output/bert2bert/preprocessed",
    logging_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir='./logs',
    warmup_steps=100,
    learning_rate=2e-5,
    # eval_accumulation_steps=2,
    predict_with_generate=True
)

In [ ]:
# test_val_dataset = Dataset.from_dict(val_dataset[0:8])

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:639: FutureWarning: Version v4.12.0 introduces a better way to train encoder-decoder models by computing the loss inside the encoder-decoder framework rather than in the decoder itself. You may observe training discrepancies if fine-tuning a model trained with versions anterior to 4.12.0. The decoder_input_ids are now created based on the labels, no need to pass them yourself anymore.
  warnings.warn(DEPRECATION_WARNING, FutureWarning)


Epoch,Training Loss,Validation Loss


In [ ]:
torch.cuda.empty_cache()
gc.collect()

2485